# Callbacks

Er zijn heel wat extra functionaliteiten die je kan toevoegen aan het trainingsproces.
De meestgebruikte zijn: 
* EarlyStopping
* ModelCheckpoint
* Tensorboard
* LearningRate schedulers

Ook kan je custom callback functions maken om je eigen functionaliteiten toe te voegen.

In de klassieke pytorch manier schrijf je je eigen trainings-lus en is het dus eenvoudig om alle gewenste code toe te voegen. 
De efficientere manier om modellen te bouwen via Keras abstraheerd echter de trainingslus en train je door middel van de fit()-methode.
Deze manier maakt het echter onmogelijk om extra functionaliteiten toe te voegen. 

Hieronder vind je een voorbeeld van hoe je callbacks kan toevoegen aan een model gebouwd met keras op een pytorch backend

In [ ]:
import numpy as np
from keras_core.models import Sequential
from keras_core.layers import Dense
from keras_core.callbacks import EarlyStopping

# maak een eenvoudig model aan
model = Sequential([
    Dense(10, activation='relu', input_shape=(10,)), # relu want hidden layer
    Dense(1) # regressie
])

model.compile(optimizer='adam', loss='mean_squared_error')

# random test data
X_train = np.random.rand(100,10)
y_train = np.random.rand(100, 1)

# CALLBACK
early_stop = EarlyStopping(monitor='val_loss', patience=2) # patience = hoeveel keer het slechter moet zijn voor het gestopt wordt

model.fit(X_train ,y_train, epochs=100, validation_split=0.2, callbacks=[early_stop])

# Dashboard for opvolgen experimenten, evaluaties, ....

Hieronder staat een voorbeelden van hoe een dashboard kan gebruikt worden met pytorch.
Ik heb hieronder gekozen voor mlflow maar alternatieven zijn tensorboard, wandb, visdom, ...

In [ ]:
!pip install mlflow

In [10]:
import mlflow
import mlflow.pytorch

import torch
import torch.nn as nn
import torch.optim as optim

# random test data
X_train = torch.randn(100,10)
y_train = torch.randn(100, 1)

# basis model, 1 hidden layer met 5 neuronen
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 1)
)

# optimizer en loss-functie definieren
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# trainingslus
with mlflow.start_run(run_name='pytorch_example'):
    for epoch in range(10):
        optimizer.zero_grad() # reset de afgeleiden
        outputs = model(X_train) # bereken de outputs
        loss = criterion(outputs, y_train) # bereken de loss
        loss.backward() # backwards propagation
        optimizer.step() # optimaliseer de gewichten

        mlflow.log_metric('loss', loss.item(), step=epoch)

    mlflow.pytorch.log_model( model, "model")

2025/10/09 11:52:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 11:52:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Door bovenstaande code uit te voeren wordt een map aangemaakt waar de nodige logs in bewaard worden.
Deze kunnen daarna gevisualiseerd worden met behulp van het volgende terminal-commando.
In de output van het commando krijg je een url te zien met de link waar je de tensorboard applicatie kan bekijken.
Met de standaardconfiguratie van de docker container moet je de applicatie kunnen bereiken via de link [localhost:6006](http://localhost:6006)

In [ ]:
!mlflow ui --host 0.0.0.0 --port 6006

/opt/conda/lib/python3.10/site-packages/mlflow/gateway/config.py:454: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Route(ConfigModel):
INFO:     Uvicorn running on http://0.0.0.0:6006 (Press CTRL+C to quit)
INFO:     Started parent process [388]
INFO:     Started server process [390]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [392]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [391]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [393]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     172.18.0.1:57016 - "GET / HTTP/1.1" 200 OK
INFO:     172.18.0.1: